# 🧠 Projet Deep Learning — Classification des Cellules Sanguines Cancéreuses (PyTorch)

## 📌 1. Contexte du Projet
Vous êtes un développeur IA junior au sein d’un laboratoire biomédical spécialisé en imagerie médicale.
Objectif : Automatiser l’analyse d’images médicales liées à deux pathologies critiques :
- Détection de **tumeurs cérébrales** (object detection à partir d’IRM),
- Classification de **cellules sanguines cancéreuses** (leucémies) à partir de frottis sanguins.

Cette première partie se concentre sur la **classification des cellules sanguines cancéreuses avec PyTorch**.

## 🧾 2. Importation des Bibliothèques Nécessaires


## 📂 3. Chargement des Images & Vérification des Extensions

- Extensions autorisées : `.jpeg`, `.jpg`, `.png`, `.bmp`
- Suppression des fichiers invalides
- Gestion des erreurs via `try-except`

## 🔎 4. Exploration des Classes du Dataset
- Liste des dossiers (classes)
- Nombre d’images par classe (`countplot`)
- Affichage d’un échantillon d’images par classe

## ✂️ 5. Division du Dataset en **Train / Validation / Test**
- Répartition : **70% / 15% / 15%**
- Vérification du nombre d’images par dossier

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(
    images, labels,
    test_size=0.3, random_state=42
)

x_val, x_test, y_val, y_test = train_test_split(
    x_test, y_test,
    test_size=0.5, random_state=42
)


base_path = r"C:\Users\issam\OneDrive\Desktop\Cancers-sanguins-Tumeurs-c-r-brales-via-Transfer-Learning\Data\Processed\Blood_Cells_Cancer"
train_path = os.path.join(base_path, "Train")
val_path = os.path.join(base_path, "Validation")
test_path = os.path.join(base_path, "Test")

for path in [train_path, val_path, test_path]:
    os.makedirs(path, exist_ok=True)

def copy_images_to_folder(image_paths, labels, destination_path):
    for img_path, label in zip(image_paths, labels):
        class_folder = os.path.join(destination_path, label)
        os.makedirs(class_folder, exist_ok=True)
        
        img_name = os.path.basename(img_path)
        dest_path = os.path.join(class_folder, img_name)
        
        try:
            shutil.copy2(img_path, dest_path)
        except Exception as e:
            print(f"Erreur lors de la copie de {img_path}: {e}")

copy_images_to_folder(x_train, y_train, train_path)

copy_images_to_folder(x_val, y_val, val_path)

copy_images_to_folder(x_test, y_test, test_path)

print("Repartition par classe :")
for dataset_name, dataset_path in [("Train", train_path), ("Validation", val_path), ("Test", test_path)]:
    print(f"\n{dataset_name}:")
    if os.path.exists(dataset_path):
        classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
        for class_name in sorted(classes):
            class_path = os.path.join(dataset_path, class_name)
            num_images = len([f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
            print(f"   - {class_name}: {num_images} images")


## 🔄 6. Data Augmentation (Seulement sur Train)
- Transformations : `blur`, `noise`, `flip`
- Objectif : équilibrer les classes et augmenter la robustesse


In [ ]:
from PIL import Image, ImageFilter
import numpy as np
import random

def add_noise(img, noise_level=20):
    arr = np.array(img)
    noise = np.random.randint(-noise_level, noise_level, arr.shape, dtype='int16')
    noisy_arr = np.clip(arr.astype('int16') + noise, 0, 255).astype('uint8')
    return Image.fromarray(noisy_arr)

def augment_image(img_path, save_dir, augmentations):
    img = Image.open(img_path)
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    ext = os.path.splitext(img_path)[1]
    aug_count = 0

    if 'blur' in augmentations:
        blurred = img.filter(ImageFilter.GaussianBlur(radius=2))
        blurred.save(os.path.join(save_dir, f"{base_name}_blur{ext}"))
        aug_count += 1

    if 'noise' in augmentations:
        noisy = add_noise(img)
        noisy.save(os.path.join(save_dir, f"{base_name}_noise{ext}"))
        aug_count += 1

    if 'flip' in augmentations:
        flipped = img.transpose(Image.FLIP_LEFT_RIGHT)
        flipped.save(os.path.join(save_dir, f"{base_name}_flip{ext}"))
        aug_count += 1

    return aug_count

for class_name in os.listdir(train_path):
    class_dir = os.path.join(train_path, class_name)
    if not os.path.isdir(class_dir):
        continue
    images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
    for img_file in images:
        img_path = os.path.join(class_dir, img_file)
        augment_image(img_path, class_dir, augmentations=['blur', 'noise', 'flip'])



## 🧪 7. Préparation des Données avec `ImageFolder` & `Transforms`
- Resize
- ToTensor
- Normalize

## 🚚 8. Création des DataLoaders
- Batch loading
- Shuffling

## 🧠 9. Chargement du Modèle Pré-entraîné **GoogLeNet**
- Remplacement de la partie **Fully Connected (FC)** par un `nn.Sequential`

## ⚙️ 10. Configuration de l’Entraînement
- Learning Rate
- Loss Function
- Optimizer

## 🏋️ 11. Boucle d’Entraînement


## 📊 12. Évaluation & Test du Modèle
- Accuracy / Loss
- Matrice de confusion (optionnel)

## 💾 13. Sauvegarde du Modèle

## ✅ 14. Conclusion & Observations
